In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


For training

FOR TA: if you want to validate my code, just replace DATA_DIR with the path where you save the training data. 

In [ ]:
%%shell 

PYTHONPATH="$/env/python:/content/drive/MyDrive/STAT940"

DATA_DIR="/content/drive/MyDrive/STAT940/"
OUT_DIR="/content/drive/MyDrive/STAT940/output3/"


mkdir -p ${OUT_DIR}
cp $0 ${OUT_DIR}

python -m encoder_decoder \
  --train_file ${DATA_DIR}/trn.jsonl \
  --eval_data_file ${DATA_DIR}/val.jsonl \
  --out_dir $OUT_DIR \
  --model_type facebook/bart-large \
  --model_name_or_path facebook/bart-large-mnli \
  --device 1 \
  --do_train \
  --do_eval \
  --save_total_limit 1 \
  --num_train_epochs 1 \
  --logging_steps 3000 \
  --gradient_accumulation_steps 8 \
  --train_batch_size 4 \
  --eval_batch_size 8 \
  --overwrite_out_dir \
  --max_input_length 1024 \
  --max_output_length 40 \
  --task index_with_sep \
  $@

Streaming output truncated to the last 5000 lines.
Iteration:  14% 2688/19607 [09:55<1:11:18,  3.95it/s]
Iteration:  14% 2689/19607 [09:55<1:06:55,  4.21it/s]
Iteration:  14% 2690/19607 [09:55<1:03:31,  4.44it/s]
Iteration:  14% 2691/19607 [09:55<1:01:15,  4.60it/s]
Iteration:  14% 2692/19607 [09:55<59:58,  4.70it/s]  
Iteration:  14% 2693/19607 [09:56<59:13,  4.76it/s]
Iteration:  14% 2694/19607 [09:56<58:10,  4.85it/s]
Iteration:  14% 2695/19607 [09:56<57:31,  4.90it/s]
Iteration:  14% 2696/19607 [09:56<1:10:49,  3.98it/s]
Iteration:  14% 2697/19607 [09:57<1:06:13,  4.26it/s]
Iteration:  14% 2698/19607 [09:57<1:03:19,  4.45it/s]
Iteration:  14% 2699/19607 [09:57<1:01:14,  4.60it/s]
Iteration:  14% 2700/19607 [09:57<1:00:08,  4.69it/s]
Iteration:  14% 2701/19607 [09:57<59:24,  4.74it/s]  
Iteration:  14% 2702/19607 [09:58<58:22,  4.83it/s]
Iteration:  14% 2703/19607 [09:58<57:35,  4.89it/s]
Iteration:  14% 2704/19607 [09:58<1:10:55,  3.97it/s]
Iteration:  14% 2705/19607 [09:58<1:06:34

Generate

In [ ]:
%%shell

PYTHONPATH="$/env/python:/content/drive/MyDrive/STAT940"
DATA_DIR="/content/drive/MyDrive/STAT940/"
MODEL_PATH="/content/drive/MyDrive/STAT940/output3/"

export DATA_DIR="/content/drive/MyDrive/STAT940/"
export MODEL_PATH="/content/drive/MyDrive/STAT940/output3/"
python -m generate --in_file $DATA_DIR/test.jsonl --out_file $MODEL_PATH/test_bart_greedy.jsonl --model_name_or_path $MODEL_PATH --beams 1 --max_length 40 --task index_with_sep --device 0

Save output

In [ ]:
import re
# a function to extract predictions
def clean_output(gold, predictions):
    """
    Utility function to clean generated output from BART
    """

    label = gold.replace("<eos>", "").strip()
    labels = [int(id_) for id_ in label.split()]

    # handle cases when output is empty
    if len(predictions) == 0:
        return labels, []

    preds = []
    for p in predictions[0].split():
        pos = re.findall('\\d+', p)
        if len(pos) == 1:
            preds.append(int(pos[0]))
    return labels, preds

Citation

In [1]:
@inproceedings{Basu-brahman-chaturvedi-rebart,
    title = "Is Everything in Order? A Simple Way to Order Sentences",
    author = "Somnath Basu Roy Chowdhury, Faeze Brahman and
      Snigdha Chaturvedi",
    booktitle = "Proceedings of the 2021 Conference on Empirical Methods in Natural Language Processing (EMNLP)",
    month = nov,
    year = "2021",
    publisher = "Association for Computational Linguistics",
}

SyntaxError: ignored